In [26]:
import pandas as pd
import numpy as np

In [2]:
!nvidia-smi

Fri May  3 04:01:49 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   37C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [4]:
from llama_index.core import SQLDatabase

/home/ai4109/.conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# start here

In [5]:
pwd

'/lustrefs/disk/project/lt900052-ai2414/NonOTG70B_Test'

## Load Model
(OpenThaiGPT finetuned with text to sql dataset (https://huggingface.co/datasets/gretelai/synthetic_text_to_sql))

In [6]:
model_path = '/project/lt900052-ai2414/Junior/SuperAI_LLM_FineTune/new_checkpoint_4'

In [7]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

In [8]:
# #Load resp synth model
# synthesizer_path = '/project/lt900048-ai24tn/models/openthaigpt/openthaigpt-1.0.0-13b-chat'
# llm_synthesizer = HuggingFaceLLM(
#     model_name=synthesizer_path,
#     tokenizer_name=synthesizer_path,
#     device_map="auto",
#     generate_kwargs = {'num_beams' : 3, 'no_repeat_ngram_size' : 3},
#     # model_kwargs={"load_in_8bit": True}
# )

In [9]:


llm = HuggingFaceLLM(
    model_name=model_path,
    tokenizer_name=model_path,
    device_map="auto",
    generate_kwargs = {'num_beams' : 5}
)

Loading checkpoint shards: 100%|██████████| 6/6 [02:08<00:00, 21.40s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [11]:
torch.cuda.is_available()

True

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = None

Embeddings have been explicitly disabled. Using MockEmbedding.


In [14]:
engine = create_engine("sqlite:////scratch/lt900052-ai2414/exp.db")

In [15]:
# test read df from sql
df = pd.read_sql('select * from financial_statement',engine)

In [16]:
df.head(5)

,Year,Company,Category,Market Cap(in B USD),Revenue,Gross Profit,Net Income,Earning Per Share,EBITDA,Share Holder Equity,...,Current Ratio,Debt/Equity Ratio,ROE,ROA,ROI,Net Profit Margin,Free Cash Flow per Share,Return on Tangible Equity,Number of Employees,Inflation Rate(in US)
0,2022,AAPL,IT,2066.94,394328.0,170782.0,99803.0,6.11,130541.0,50672.0,...,0.8794,2.3695,196.9589,28.2924,66.6994,25.3096,1.3146,196.9589,164000,8.0028
1,2021,AAPL,IT,2913.28,365817.0,152836.0,94680.0,5.61,120233.0,63090.0,...,1.0746,1.9768,150.0713,26.9742,54.9839,25.8818,1.3261,150.0713,154000,4.6979
2,2020,AAPL,IT,2255.97,274515.0,104956.0,57411.0,3.28,77344.0,65339.0,...,1.3636,1.7208,87.8664,17.7256,35.0054,20.9136,1.0183,87.8664,147000,1.2336
3,2019,AAPL,IT,1304.76,260174.0,98392.0,55256.0,2.97,76477.0,90488.0,...,1.5401,1.1940,61.0645,16.3230,30.3113,21.2381,-0.0388,61.0645,137000,1.8122
4,2018,AAPL,IT,748.54,265595.0,101839.0,59531.0,2.98,81801.0,107147.0,...,1.1329,1.0685,55.5601,16.2775,29.6348,22.4142,0.7414,55.5601,132000,2.4426


In [16]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="models/bge-m3")

In [18]:
from llama_index.core import Settings
Settings.embed_model = embed_model

In [35]:
table_name = "customer_support_tickets.faiss"

In [37]:
import faiss
from llama_index.readers.faiss import FaissReader
index = faiss.read_index("/scratch/lt900052-ai2414/"+table_name)
reader = FaissReader(index)


In [42]:
search_text = 'สัดส่วนของลูกค้าชายที่ซื้อ Macbook'
search_vector = embed_model.get_text_embedding(search_text)
_vector = np.array([search_vector])

In [43]:
distances, ann = index.search(_vector, k=3)

In [55]:
import json
d = dict()
with open('/scratch/lt900052-ai2414/customer_support_tickets.json') as json_data:
    d = json.load(json_data)


In [60]:
rag_examples = [d[str(x)] for x in ann[0]]

In [64]:
rag_examples

[["SELECT AVG(Customer_Age) FROM customer_support_tickets WHERE Product_Purchased = 'Macbook';",
  "หา Customer_Age เฉลี่ยของลูกค้าที่ซื้อ 'Macbook'"],
 ["SELECT COUNT(*) FROM customer_support_tickets WHERE Customer_Age BETWEEN 25 AND 35 AND Product_Purchased = 'MacBook';",
  "มี Customer_Age จำนวนเท่าไหร่ ที่อยู่ในช่วง 25-35 ปี และ Product_Purchased เป็น 'MacBook'"],
 ['SELECT "Product_Purchased", COUNT(*) AS "Total_Purchased" FROM "customer_support_tickets" WHERE "Customer_Gender" = \'Male\' GROUP BY "Product_Purchased" ORDER BY "Total_Purchased" DESC LIMIT 1; ',
  'ลูกค้าชายซื้ออะไรมากที่สุด']]

In [65]:
formatted_strings = []
for i, example in enumerate(rag_examples, start=1):
    formatted_string = f"Question: {example[1]}\nSql query: {example[0]}"
    formatted_strings.append(formatted_string)

formatted_output = "\n".join(formatted_strings)

In [32]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
sql_database = SQLDatabase(engine, include_tables=["online_shopping",])
#'financial_statement','spotify','customer_support_tickets'

INFO:faiss.loader:Loading faiss with AVX2 support.
Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.


In [18]:
# # TABLE B Prompt
# new_prompt_str = """<s>[INST] <<SYS>>\nYou are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>\n\n
# You are a SQLite expert. Given an input question,create a syntactically correct SQLite query to run.
# You can order the results to return the most informative data in the database.
# You are working with only one table named financial_statement.

# DO WRAP EVERY COLUMN NAME WITH (").
# For example: DO: "Market Cap".
# DO NOT: Market Cap
# Use the following format:

# SQLQuery: SQL Query to run

# Only use the following columns of the financial_statement table:
# Year
# Description: Year of financial statement

# Data Type: Categorical;

# Company
# ชื่อบริษัท
# Data Type: Text;

# Category
# หมวดหมู่
# Data Type: Categorical;

# Market Cap(in B USD)
# มูลค่าตลาด (พันล้าน USD)
# Data Type: Numeric;

# Revenue
# Data Type: Numeric;

# Gross Profit
# ผลกำไรขั้นต้น
# Data Type: Numeric;

# Net Income
# ผลกำไรสุทธิ
# Data Type: Numeric;

# Earning Per Share
# กำไรต่อหุ้น
# Data Type: Numeric;

# EBITDA:
# กำไรก่อนดอกเบี้ย ภาษี ค่าเสื่อมราคาและค่าตัดจำหน่าย
# Data Type: Numeric;

# Share Holder Equity
# ส่วนของผู้ถือหุ้น
# Data Type: Numeric;

# Cash Flow from Operating
# Data Type: Numeric;

# Cash Flow from Investing
# Description: Any inflows or outflows of cash from a company's long-term investments.
# กระแสเงินสดจากการลงทุน
# Data Type: Numeric;

# Cash Flow from Financial Activities
# กระแสเงินสดจากกิจกรรมจัดหาเงิน
# Data Type: Numeric;

# Current Ratio
# Data Type: Date;

# Debt/Equity Ratio
# อัตราส่วน หนี้สิน ต่อ ทุน
# Data Type: Numeric;

# ROE
# ผลตอบแทนต่อส่วนของผู้ถือหุ้น
# Data Type: Numeric;

# ROA
# อัตราผลตอบแทนจากสินทรัพย์ทั้งหมด
# Data Type: Numeric;

# ROI
# ผลตอบแทนจากการลงทุนนั่นเอง.
# Data Type: Numeric;

# Net Profit Margin
# อัตรากำไร (%)
# Data Type: Numeric;

# Free Cash Flow per Share

# Data Type: Numeric;

# Return on Tangible Equity

# Data Type: Numeric;

# Number of Employees
# Description: Number of Employee in each company
# จำนวนพนักงาน
# Data Type: Numeric;

# Inflation Rate(in US)
# Description: the rate of increase in prices over a given period of time in US
# อัตราเงินเฟ้อ
# Data Type: Numeric;

# Here's some example rows from the table:
# <Table>
# Year	Company	Category	Market Cap(in B USD)	Revenue	Gross Profit	Net Income	Earning Per Share	EBITDA	Share Holder Equity	Cash Flow from Operating	Cash Flow from Investing	Cash Flow from Financial Activities	Current Ratio	Debt/Equity Ratio	ROE	ROA	ROI	Net Profit Margin	Free Cash Flow per Share	Return on Tangible Equity	Number of Employees	Inflation Rate(in US)
# MSFT	IT	238.78	62484	50089	18760	2.1	26771	46175	24073	-11314	-13291	2.1293	0.1286	40.628	21.7853	36.7023	30.0237	0.7057	57.5054	94000	1.64
# </Table>
# ###RULES
# Remember to DO WRAP EVERY COLUMN NAME WITH single quote(").
# For example: DO: "Market Cap".
# DO NOT: Market Cap
# And DO NOT CHANGE Company symbol to it's full name
# for example: MSFT should stay as MSFT. NEVER Microsoft
# IF the question is not related to the columns or table. Just say I don't know.
# This is some relevant question and their SQL examples:
# Question:บริษัท MSFT จัดอยู่ใน Category อะไร ในปี 2022
# SQL Query:SELECT Category FROM financial_statement WHERE Year = 2022 AND Company = "MSFT" 
# Question:ผลรวมกำไรขั้นต้นของบริษัท XXX 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) มากกว่า ผลรวมกำไรขั้นต้นของบริษัท YYY 5 ปีย้อนหลัง (ตั้งแต่ปี 2019-2023) หรือไม่ กรุณาตอบเป็น 'True' หรือ 'False',
# SQL Query:SELECT SUM(CASE WHEN a.Company = 'XXX' THEN a."Gross Profit" ELSE 0 END) AS XXX_Gross_Profit, SUM(CASE WHEN a.Company = 'YYY' THEN a."Gross Profit" ELSE 0 END) AS YYY_Gross_Profit FROM (SELECT * FROM financial_statement WHERE Company IN ('XXX', 'YYY') AND Year BETWEEN 2019 AND 2023) a;
# Question: {query_str}
# SQL Query: 
# [/INST]"""

In [19]:
# new_prompt_str = """<<SYS>>\nYou are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>\n\n
# You are a SQLite expert. Given an input question,create a syntactically correct SQLite query to run.
# You can order the results to return the most informative data in the database.
# Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
# Pay attention to use only the column names you can see in the table below. Be careful to not query for columns that do not exist.
# You are working with only one table named online_shopping.
# Unless the query specifies the number of results, you MUST give only 1 answer, using LIMIT 1
# DO WRAP EVERY COLUMN NAME WITH (").
# For example: DO: "Market Cap".
# DO NOT: Market Cap
# Use the following format:

# SQLQuery: SQL Query to run


# Only use the following columns of the online_shopping table:
# ### CustomerID
# รหัสลูกค้า

# ### Gender
# Description: Gender of the customer (e.g., M,F).
# เพศ

# ### Location
# สถานที่ เมือง
# Data Type: Text;
# ### Tenure_Months

# ### Transaction_ID

# ### Transaction_Date
# วันที่ทำธุรกรรม

# ### Product_SKU
# รหัสสินค้า

# ### Product_Description

# ### Product_Category:
# ประเภทสินค้า

# ### Quantity
# จำนวนสินค้าที่ซื้อ

# ### Avg_Price
# ราคาเฉลี่ย

# ### Total_Price
# ราคารวม ไม่รวมค่าจัดส่ง

# ### Delivery_Charges
# ค่าจัดส่ง

# ### Date
# วันที่

# ### Month
# เดือน

# Here's some example rows from the table:
# <Table>
# CustomerID	Gender	Location	Tenure_Months	Transaction_ID	Transaction_Date	Product_SKU	Product_Description	Product_Category	Quantity	Avg_Price	Total_Price	Delivery_Charges	Date	Month
# 17850	M	Chicago	12	16679	2019-01-01	GGOENEBJ079499	Nest Learning Thermostat 3rd Gen-USA - Stainless Steel	Nest-USA	1	153.71	153.71	6.5	1/1/2019	1
# 17850	M	Chicago	12	16680	2019-01-01	GGOENEBJ079499	Nest Learning Thermostat 3rd Gen-USA - Stainless Steel	Nest-USA	1	153.71	153.71	6.5	1/1/2019	1
# 17850	M	Chicago	12	16696	2019-01-01	GGOENEBQ078999	Nest Cam Outdoor Security Camera - USA	Nest-USA	2	122.77	245.54	6.5	1/1/2019	1
# 17850	M	Chicago	12	16699	2019-01-01	GGOENEBQ079099	Nest Protect Smoke + CO White Battery Alarm-USA	Nest-USA	1	81.5	81.5	6.5	1/1/2019	1
# 17850	M	Chicago	12	16700	2019-01-01	GGOENEBJ079499	Nest Learning Thermostat 3rd Gen-USA - Stainless Steel	Nest-USA	1	153.71	153.71	6.5	1/1/2019	1
# </Table>

# ###RULES
# Remember to DO WRAP EVERY COLUMN NAME WITH single quote(").
# For example: DO: "Market Cap".
# DO NOT: Market Cap


# Question: {query_str}
# SQL Query:[/INST]
# """

In [67]:
#Table C prompt
new_prompt_str = f"""<<SYS>>\nYou are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>\n\n
You are a SQLite expert. Given an input question,create a syntactically correct SQLite query to run.
You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the table below. Be careful to not query for columns that do not exist.
You are working with only one table named spotify.
Unless the query specifies the number of results, you MUST give only 1 answer, using LIMIT 1
DO WRAP EVERY COLUMN NAME WITH (").
For example: DO: "Market Cap".
DO NOT: Market Cap
Use the following format:

SQLQuery: SQL Query to run


Only use the following columns of the spotify table:
Table name : spotify
    Column : "track_name"
    respresent: Name of the song
    Data Type: Text;

    Column : "artist(s)_name"
    respresent: Name of the artist(s) of the song
    Data Type: Text;

    Column : "artist_count"
    respresent: Number of artists contributing to the song
    Data Type: [1, 2, 3, 4, 5, 6, 7, 8]

    Column : "released_year"
    respresent: Year when the song was released
    Data Type: [2023, 2019, 2022, 2013, 2014, 2018, 2017, 2020, 2016, 2012, 1999,
       2008, 1975, 2021, 2015, 2011, 2004, 1985, 2007, 2002, 2010, 1983,
       1992, 1968, 1984, 2000, 1997, 1995, 2003, 1973, 1930, 1994, 1958,
       1957, 1963, 1959, 1970, 1971, 1952, 1946, 1979, 1950, 1942, 1986,
       2005, 1991, 1996, 1998, 1982, 1987]

    Column : "released_month"
    respresent: Month when the song was released
    Data Type: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

    Column : "released_day"
    respresent: Day of the month when the song was released
    Data Type: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
    19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

    Column : in_spotify_playlists
    respresent: Number of Spotify playlists the song is included in
    Data Type: Numeric;

    Column : in_spotify_charts
    respresent: Presence and rank of the song on Spotify charts
    Data Type: Numeric;

    Column : streams
    respresent: Total number of streams on Spotify
    Data Type: object;

    Column : in_apple_playlists
    respresent: Number of Apple Music playlists the song is included in
    Data Type: Numeric;

    Column : in_apple_charts
    respresent: Presence and rank of the song on Apple Music charts
    Data Type: Numeric;

    Column : in_deezer_playlists
    respresent: Number of Deezer playlists the song is included in
    Data Type: [10, 14, 12, 15, 17, 13, 11,  5, 58, 24, 46,  6,  2,  4,  1, 38,  8,
       23,  9, 18,  7,  3, 45, 16, 37,  0, 19, 20, 28, 26, 31, 22, 29, 21]

    Column : in_deezer_charts
    respresent: Presence and rank of the song on Deezer charts
    Data Type: Numeric;

    Column : in_shazam_charts
    respresent: Presence and rank of the song on Shazam charts
    Data Type: object;

    Column : bpm
    respresent: Beats per minute, a measure of song tempo
    Data Type: Numeric;

    Column : key
    respresent: Key of the song
    Data Type: ['B', 'C#', 'F', 'A', 'D', 'F#', nan, 'G#', 'G', 'E', 'A#', 'D#']

    Column : mode
    respresent: Mode of the song (major or minor)
    Data Type: Categorical; ['Major', 'Minor']

    Column : danceability_%
    respresent: Percentage indicating how suitable the song is for dancing
    Data Type: Numeric;

    Column : valence_%
    respresent: Positivity of the song's musical content
    Data Type: Numeric;

    Column : energy_%
    respresent: Perceived energy level of the song
    Data Type: Numeric;

    Column : acousticness_%
    respresent: Amount of acoustic sound in the song
    Data Type: Numeric;

    Column : instrumentalness_%
    respresent: Amount of instrumental content in the song
    Data Type: Numeric;

    Column : liveness_%
    respresent: Presence of live performance elements
    Data Type: Numeric;

    Column : speechiness_%
    respresent: Amount of spoken words in the song
    Data Type: Numeric;

Here's some example rows from the table:
<Table>
track_name	artist(s)_name	artist_count	released_year	released_month	released_day	in_spotify_playlists	in_spotify_charts	streams	in_apple_playlists	in_apple_charts	in_deezer_playlists	in_deezer_charts	in_shazam_charts	bpm	key	mode	danceability_%	valence_%	energy_%	acousticness_%	instrumentalness_%	liveness_%	speechiness_%
Seven (feat. Latto) (Explicit Ver.)	Latto, Jung Kook	2	2023	7	14	553	147	141381703	43	263	45	10	826	125	B	Major	80	89	83	31	0	8	4
LALA	Myke Towers	1	2023	3	23	1474	48	133716286	48	126	58	14	382	92	C#	Major	71	61	74	7	0	10	4
vampire	Olivia Rodrigo	1	2023	6	30	1397	113	140003974	94	207	91	14	949	138	F	Major	51	32	53	17	0	31	6
Cruel Summer	Taylor Swift	1	2019	8	23	7858	100	800840817	116	207	125	12	548	170	A	Major	55	58	72	11	0	11	15
</Table>

###RULES
Remember to DO WRAP EVERY COLUMN NAME WITH single quote(").
For example: DO: "Market Cap".
DO NOT: Market Cap

Here's some revelant question and query for example:
{formatted_output}

"""
new_prompt_str += "Question: {query_str}\nSQL Query:[/INST]"
new_prompt_str

'<<SYS>>\nYou are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>\n\n\nYou are a SQLite expert. Given an input question,create a syntactically correct SQLite query to run.\nYou can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the table below. Be careful to not query for columns that do not exist.\nYou are working with only one table named spotify.\nUnless the query specifies the number of results, you MUST give only 1 answer, using LIMIT 1\nDO WRAP EVERY COLUMN NAME WITH (").\nFor example: DO: "Market Cap".\nDO NOT: Market Cap\nUse the following format:\n\nSQLQuery: SQL Query to run\n\n\nOnly use 

In [21]:
# #Table D prompt
# new_prompt_str = """<<SYS>>\nYou are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>\n\n
# You are a SQLite expert. Given an input question,create a syntactically correct SQLite query to run.
# You can order the results to return the most informative data in the database.
# Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
# Pay attention to use only the column names you can see in the table below. Be careful to not query for columns that do not exist.
# You are working with only one table named customer_support_tickets.
# Unless the query specifies the number of results, you MUST give only 1 answer, using LIMIT 1
# DO WRAP EVERY COLUMN NAME WITH (").
# For example: DO: "Market Cap".
# DO NOT: Market Cap
# Use the following format:

# SQLQuery: SQL Query to run


# Only use the following columns of the customer_support_tickets table:
# Table name : "customer_support_tickets"

#     Column: "Ticket_ID"
#     respresent: A unique identifier for each ticket.
#     values : Numeric;

#     Column: "Customer_Email"
#     respresent: The email address of the customer (Domain name - @example.com is intentional for user data privacy concern).
#     values : Text;

#     Column: "Customer_Age"
#     respresent: The age of the customer.
#     values : Numeric;

#     Column: "Customer_Gender"
#     respresent: The gender of the customer.
#     values : ['Male','Female','Other']

#     Column: "Product_Purchased"
#     respresent: The tech product purchased by the customer.
#     values : Text;

#     Column: "Date_of_Purchase"
#     respresent: The date when the product was purchased.
#     values : Year-Month-Day;

#     Column: "Ticket_Type"
#     respresent: The type of ticket (e.g., technical issue, billing inquiry, product inquiry).
#     values : ['Refund request','Technical issue','Cancellation request','Product inquiry','Billing inquiry']

#     Column: "Ticket_Subject"
#     respresent: The subject/topic of the ticket.
#     values : ['Refund request','Software bug','Product compatibility','Delivery problem','Hardware issue','Battery life',
#     Network problem','Installation support','Product setup','Payment issue','Product recommendation','Account access','Peripheral compatibility',
#     'Data loss','Cancellation request','Display issue']

#     Column: "Ticket_Description"
#     respresent: The description of the customer's issue or inquiry.
#     values : Text;

#     Column: "Ticket_Status"
#     respresent: The status of the ticket (e.g., open, closed, pending customer response).
#     values : ['Pending Customer Response','Open','Closed']

#     Column: "Resolution"
#     respresent: The resolution or solution provided for closed tickets.
#     values : Text;

#     Column: "Ticket_Priority"
#     respresent: The priority level assigned to the ticket (e.g., low, medium, high, critical).
#     values : ['Medium','Critical','High','Low']

#     Column: "Ticket_Channel"
#     respresent: The channel through which the ticket was raised (e.g., email, phone, chat, social media).
#     values : ['Email','Phone','Social media','Chat']

#     Column: "First_Response_Time"
#     respresent: The time taken to provide the first response to the customer.
#     values : Year-Month-Day Hour:minute:second;

#     Column: "Time_to_Resolution"
#     respresent: The time taken to resolve the ticket.
#     values : Year-Month-Day Hour:minute:second;

#     Column: "Customer_Satisfaction_Rating"
#     respresent: The customer's satisfaction rating for closed tickets (on a scale of 1 to 5).

# Here's some example rows from the table:
# <Table>
# Ticket_ID	Customer_Name	Customer_Email	Customer_Age	Customer_Gender	Product_Purchased	Date_of_Purchase	Ticket_Type	Ticket_Subject	Ticket_Description	Ticket_Status	Resolution	Ticket_Priority	Ticket_Channel	First_Response_Time	Time_to_Resolution	Customer_Satisfaction_Rating
# 1	Marisa Obrien	carrollallison@example.com	32	Other	GoPro Hero	2021-03-22	Technical issue	Product setup	"I'm having an issue with the {product_purchased}. Please assist.

# Your billing zip code is: 71701.

# We appreciate that you have requested a website address.

# Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists."	Pending Customer Response		Critical	Social media	2023-06-01 12:15:36		
# 2	Jessica Rios	clarkeashley@example.com	42	Female	LG Smart TV	2021-05-22	Technical issue	Peripheral compatibility	"I'm having an issue with the {product_purchased}. Please assist.

# If you need to change an existing product.

# I'm having an issue with the {product_purchased}. Please assist.

# If The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly."	Pending Customer Response		Critical	Chat	2023-06-01 16:45:38		
# 3	Christopher Robbins	gonzalestracy@example.com	48	Other	Dell XPS	2020-07-14	Technical issue	Network problem	"I'm facing a problem with my {product_purchased}. The {product_purchased} is not turning on. It was working fine until yesterday, but now it doesn't respond.

# 1.8.3 I really I'm using the original charger that came with my {product_purchased}, but it's not charging properly."	Closed	Case maybe show recently my computer follow.	Low	Social media	2023-06-01 11:14:38	2023-06-01 18:05:38	3
# 4	Christina Dillon	bradleyolson@example.org	27	Female	Microsoft Office	2020-11-13	Billing inquiry	Account access	"I'm having an issue with the {product_purchased}. Please assist.

# If you have a problem you're interested in and I'd love to see this happen, please check out the Feedback. I've already contacted customer support multiple times, but the issue remains unresolved."	Closed	Try capital clearly never color toward story.	Low	Social media	2023-06-01 07:29:40	2023-06-01 01:57:40	3
# </Table>

# ###RULES
# Remember to DO WRAP EVERY COLUMN NAME WITH single quote(").
# For example: DO: "Market Cap".
# DO NOT: Market Cap


# Question: {query_str}
# SQL Query:[/INST]
# """

In [22]:
# resp_synthesis_prompt_str = ("""
# [INST]<<SYS>>
# You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>
# "The original query is as follows: {query_str}\n"
#     "We have provided an existing answer: {context_str}\n"
#     "You must refine the existing answer to match the instruction in the original query"
#     "If the context isn't useful, return the original answer.\n"
#     YOU MUST NEVER USE ANY INFORMATION OUTSIDE OF THE EXISTING ANSWER
#     "If there's no additional instruction in the original query, just return the existing answer"
#     For example:
#     Original Query: ผู้หญิงหรือผู้ชายใครมียอดจำนวน record การสั่งซื้อมากกว่ากัน กรุณาตอบแค่ `ผู้ชาย` หรือ `ผู้หญิง`
#     Existing answer: ['ผู้หญิง']
#     Refined Answer: ผู้หญิง

#     Original Query: ในเดือนตุลาคมผู้หญิงที่ California นิยมซื้อ Product SKU ไหนมากที่สุด
#     Existing answer: [('GGOENEBQ078999', 168)]
#     Refined Answer: GGOENEBQ078999
    
#     Original Query: {query_str}
#     Existing answer: {context_str}
#     "Refined Answer:[/INST]"
# """)

In [23]:
resp_synthesis_prompt_str = ("""[INST]<<SYS>>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>

Refine the given answer so that it directly addresses the original question:
Do not explain your answer.
DO not any information other than the answer.
Original question: {query_str}
Answer: {context_str}
New_Answer: [/INST]
""")

In [24]:
from llama_index.core import PromptTemplate
new_prompt = PromptTemplate(new_prompt_str)
resp_synthesis_prompt = PromptTemplate(resp_synthesis_prompt_str)

In [25]:
from llama_index.core.retrievers import NLSQLRetriever
sql_retriever = NLSQLRetriever(text_to_sql_prompt = new_prompt, sql_database = sql_database,tables = ['spotify'], llm = llm)

In [26]:
node = sql_retriever.retrieve("CustomerID ที่มียอดจำนวนเงินสั่งซื้อไม่รวมค่าขนส่งน้อยที่สุดมาจากที่เมืองไหน")
node

Token indices sequence length is longer than the specified maximum sequence length for this model (2256 > 2048). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[NodeWithScore(node=TextNode(id_='3ff68856-7064-4d2e-a66f-a6fa8b1dafce', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Error: Statement 'SELECT C.City FROM Customers C INNER JOIN (SELECT CustomerID, MIN(TotalAmount) AS MinTotalAmount FROM Orders GROUP BY CustomerID) O ON C.CustomerID = O.CustomerID WHERE O.MinTotalAmount = (SELECT MIN(TotalAmount) FROM Orders);' is invalid SQL.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)]

In [27]:
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(
    response_mode="compact",
    llm = llm,
    text_qa_template=resp_synthesis_prompt,
    refine_template=resp_synthesis_prompt,
    use_async=False,
    streaming=False,
)

In [28]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core import Response
class EXPQueryEngine(CustomQueryEngine):
    """EXP Query Engine."""

    retriever: NLSQLRetriever
    response_synthesizer: BaseSynthesizer

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        response_obj = Response(nodes[0].text)
        return response_obj

In [29]:
EXP_engine = EXPQueryEngine(
    retriever = sql_retriever,
    response_synthesizer = response_synthesizer
)

In [32]:

query_str = """เพลย์ลิสต์ Deezer มีกี่เพลง?"""
response = EXP_engine.query(query_str)



NameError: name 'EXP_engine' is not defined

In [31]:

response

Response(response='[(46,)]', source_nodes=[], metadata=None)